# Cars Object Detection Demo
Welcome to the cars object detection inference walkthrough! This notebook will walk you step by step through the process of using a pre-trained model to detect cars in an image. 

Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

## 1. Data Preparation (Option) 
### (1) Download images from web
The train dataset is already in the floder ~/Quick_Start_Demo/labelcar/images/. There are 6 images including CAR_1.jpg, CAR_2.jpg, …, CAR_6.jpg. The trainval.txt file is already in the floder ~/labelcar/annotations/, and its format is "image_name class_id" such as 

CAR_1 1 

CAR_2 1

...

CAR_6 1

Also, you can download the whole [cars datasets](https://ai.stanford.edu/~jkrause/cars/car_dataset.html).

### (2) Collect images by your camera
### (3) Extract images from your videos 

## 2. Label Images by LabelImg Toolkit
Use annotation tool to label images. We use labelImg in this demo. First of all, invoking the labelImg by running the next command line.

In [ ]:
!python3 ../Annotation_Tool/labelImg/labelImg.py

Step 1: Use the "Open Dir", and choose the directory ~/Quick_Start_Demo/labelcar/images/.

Step 2: Use the "Change Save Dir", and choose the directory ~/Quick_Start_Demo/labelcar/annotations/xmls/.

Step 3: Use "PascalVOC" format.

Step 4: Use "Edit">>"Create\nRectBox", 
![jupyter](./jupyter_images/1.png)

Step 5: Draw the car region. Then select the label "car", and Use "Save" after labeling all the cars in an image.
![jupyter](./jupyter_images/2.png)

Step 6: Use "Next Image", and go back to step4 and step5.

Step 7: After label all 6 images, close labelImg.

## 3. Create TF Records
The car_label_map.pbtxt file is already in the floder ~/quick_start_object_detection/object_detection/labelcar/. It has class_id and class_name such as

item {

  id: 1
  
  name: 'CAR'

}

Now use create_sw_tf_record.py to create tensorflow records.

In [ ]:
!python3 create_tf_record.py --label_map_path=./object_detection/labelcar/car_label_map.pbtxt --data_dir=./labelcar --output_dir=./object_detection/labelcar

i## 4. Training  Model
### (1) Download a pre-trained model (Option)
The pre-trained model ssd_inception_v2_coco.ckpt is already in the floder ~/quick_start_object_detection/Incepion_v2_coco/. 

You can download other models from [tensorflow object detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).

### (2) Create a configuration file (Option)
The configuration file ssd_inception_v2_pets.config is already in the floder ~/quick_start_object_detection/object_detection/labelcar/. You can modify the parameters in this file. The default parameters of the file are 

num_classes: 1 in line 9

fine_tune_checkpoint path in line 153

num_steps: 520 in line159

input_path in line 172 & 186

label_map_path in line 174 & 188

NOTE: If you use the other model, you should choose the corresponding [configuration file](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) to your model.

### (3) Start training

clone_on_cpu=True means training by CPU. Default option.


In [ ]:
!python3 train.py --clone_on_cpu=True --logtostderr --train_dir=./object_detection/labelcar --pipeline_config_path=./object_detection/labelcar/ssd_inception_v2_pets.config --label_map_path=./object_detection/labelcar/car_label_map.pbtxt 

clone_on_cpu=False means training by Nvidia GPU, use this one if there is Nvidia GPU available

In [ ]:
#!python3 train.py --clone_on_cpu=False --logtostderr --train_dir=./object_detection/labelcar --pipeline_config_path=./object_detection/labelcar/ssd_inception_v2_pets.config --label_map_path=./object_detection/labelcar/car_label_map.pbtxt 

### (4) Visualizing by tensorboard (Option)
You can see the loss curve and so on by tensorboard. Open a new command prompt or terminal, run the following command line:

cd ~/Quick_Start_Demo/

tensorboard --logdir=./object_detection/labelcar

Now open a Explorer, and visit http://localhost:6006

## 5. Export The Frozen Graph
Convert the ckpt model to pb model.

In [ ]:
!python3 export_inference_graph.py --trained_checkpoint_prefix ./object_detection/labelcar/model.ckpt-520 --input_type image_tensor --pipeline_config_path ./object_detection/labelcar/ssd_inception_v2_pets.config --output_directory ./car_inference_graph

## 6. Test Your Model on Images

### Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Object detection imports
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

### Model preparation 

#### Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.

In [ ]:
# What model to download.
MODEL_NAME = 'car_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = './' + MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('./object_detection/labelcar', 'car_label_map.pbtxt')

NUM_CLASSES = 1

# Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

### Detection in images

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = './labelcar/test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 4) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

#Add below to fix issue with GPU initialization
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with detection_graph.as_default():
    with tf.Session(graph=detection_graph, config=config) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)